# Create New Cycle

This notebook creates a new active cycle for your Risk Modeling workflow. 

1. Archives the current active cycle (if any)
2. Creates a new cycle from the template
3. Registers stages and steps in the database

**Note:** Only one cycle can be active at a time.

In [ ]:
# Import modules directly
from system import cycle, database as db, ux
from system.context import WorkContext
from system.step import Step

## Cycle Name

In [2]:
# Modify this to your desired cycle name
cycle_name = "Q4_2025_Demo-Analysis"

## Look into current cycles (if any)

In [ ]:
ux.subheader("Current Cycles")

cycles_df = cycle.get_cycle_status()
if not cycles_df.empty:
    ux.dataframe(cycles_df, max_rows=10)
else:
    print("No cycles found")

# Check for active cycle
active_cycle = db.get_active_cycle()
if active_cycle:
    print(f"\nActive cycle exists: {active_cycle['cycle_name']}")
    print("This will be archived when creating a new cycle")
    raise Warning("Active cycle detected. Please review before creating a new cycle.")
else:
    ux.success("No active cycle - ready to create new cycle")

No cycles found


In [ ]:
ux.subheader("Validating Cycle Name")
print(f"Proposed cycle name: {cycle_name}")

# Validate the name
valid, message = cycle.validate_cycle_name(cycle_name)

if valid:
    ux.success(message)
else:
    ux.error(f"\nPlease modify the cycle_name above and re-run this cell. {message}")
    raise RuntimeError(message)
    

Proposed cycle name: Q4_2025_Demo-Analysis


In [ ]:
# Creating the cycle
if valid:
    ux.header(f"Creating Cycle: {cycle_name}")
    
    # Create the cycle
    success = cycle.create_cycle(cycle_name)
    
    if success:
        ux.header("CYCLE CREATED SUCCESSFULLY")
        
        # Show the new cycle
        new_cycle = db.get_active_cycle()
        if new_cycle:
            ux.cycle_summary(new_cycle)
        
        # Show directory structure
        from pathlib import Path
        active_dir = Path(f"home/jovyan/workspace/workflows/Active_{cycle_name}")
        
        if active_dir.exists():
            ux.subheader("Directory Structure")
            print(f"📁 {active_dir}\n")
            
            for item in sorted(active_dir.iterdir()):
                if item.is_dir():
                    print(f"  📂 {item.name}/")
                    if item.name == 'notebooks':
                        for subitem in sorted(item.iterdir()):
                            if subitem.is_dir():
                                print(f"      📂 {subitem.name}/")
    else:
        ux.error("Failed to create cycle")
else:
    ux.warning("Please fix the cycle name and try again")


Creating cycle: Q4_2025_Demo-Analysis
Created directory: /home/jovyan/workspace/workflows/Active_Q4_2025_Demo-Analysis
Registered 1 stages with steps

Cycle 'Q4_2025_Demo-Analysis' created successfully


Property,Value
Status,active
Created,2025-10-10 17:44:24.341276+00:00
Created By,notebook_user


In [ ]:
ux.header("Verification")

# Check database
active_cycle = db.get_active_cycle()
if active_cycle:
    ux.success(f"Active cycle in database: {active_cycle['cycle_name']}")
else:
    ux.error("No active cycle in database")

# Check filesystem
from pathlib import Path
workflows_path = Path('/home/jovyan/workspace/workflows')
active_dirs = [d for d in workflows_path.iterdir() if d.is_dir() and d.name.startswith('Active_')]

if active_dirs:
    ux.success(f"Active directory found: {active_dirs[0].name}")
else:
    ux.error("No active directory found")

# Show registered steps
if active_cycle:
    progress_df = cycle.get_cycle_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        ux.subheader(f"The cycle has {len(progress_df)} steps")
        ux.step_progress(progress_df)
    else:
        ux.warning("No stages and steps - template may be empty")

---